# **The Chat Format**

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [28]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [30]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
load_dotenv("OpenAI.env") # read local .env file
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


In [32]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content


def get_completion_from_messages(message, model="gpt-3.5-turbo", temperature=0): 
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content

In [34]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [36]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side, aye, a classic jest from days of yore!


In [38]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hi Isa! It's nice to meet you. How can I assist you today?


In [40]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but I'm not able to know your name unless you tell me. How about you share your name with me?


In [42]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa.


# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [45]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


In [47]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

In [54]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "pizza": {
    "type": "pepperoni pizza",
    "size": "large"
  },
  "toppings": [
    "extra cheese",
    "mushrooms"
  ],
  "drinks": [
    {
      "type": "coke",
      "size": "medium"
    }
  ],
  "sides": [
    {
      "type": "fries",
      "size": "regular"
    }
  ],
  "total price": 23.45
}


## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

In [60]:

# Initialize context and panels
panels = []  # collect display 
context = [ 
    {'role':'system', 'content':"""
    You are GalacticOrderBot, an intergalactic pizza service that serves customers in outer space. 
    Greet the customer with a space-themed greeting and take their order for space pizzas.
    The menu includes:
    - Space Pizza 15.99, 13.99, 11.99
    - Meteorite Pizza 16.99, 14.99, 12.99
    - Galactic Veggie 14.99, 12.99, 10.99
    - Alien Mushrooms $2.50, Meteorite Crust $3.00, Star Cheese $2.00
    - Galactic Sausage $3.50, Interstellar Peppers $1.50
    Drinks:
    - Galactic Soda $5.00, $4.00, $3.00
    - Cosmic Water $2.00
    - Lunar Juice $4.00, $3.00, $2.00
    Sides:
    - Comet Fries $5.00, $3.00
    - Stardust Salad $7.00

    Please be creative in your responses and make sure the customer enjoys their space-themed pizza experience!
    """}
]

# Input field and button for conversation
inp = pn.widgets.TextInput(value="Hello, GalacticOrderBot!", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Send Order!")
interactive_conversation = pn.bind(lambda _: collect_messages(), button_conversation)

# Main GalacticOrderBot function for collecting the order
def collect_messages():
    prompt = inp.value
    inp.value = ''  # Reset the input field
    context.append({'role': 'user', 'content': f"{prompt}"})  # Add user's input to context
    response = get_completion_from_messages(context)  # Get the bot's response
    context.append({'role': 'assistant', 'content': f"{response}"})  # Add assistant's response to context

    # Convert the response dictionary to a formatted string
    formatted_response = json.dumps(response, indent=4)  # Convert JSON response to string format

    # Display user and assistant's messages in the interface
    panels.append(pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(pn.Row('Assistant:', pn.pane.Markdown(formatted_response, width=600, styles={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)  # Return the updated panel

# JSON generation of the order summary after collecting the complete order
def generate_order_summary():
    messages = context.copy()  # Copy the context for the final request
    messages.append({
        'role': 'system', 
        'content': """
        Create a JSON summary of the previous food order. Itemize the price for each item:
        1) pizza, include size and price 
        2) list of toppings 
        3) list of drinks, include size and price 
        4) list of sides, include size and price
        5) total price
        """
    })

    response = get_completion_from_messages(messages, temperature=0)  # Get the JSON summary from the assistant
    print(response)  # Print the final summary (this will be returned in the real app)
    return response

# Function that simulates getting a response from the assistant (Replace with actual response function)
def get_completion_from_messages(messages, temperature=0):
    # Here, you can plug in the actual AI response mechanism. For now, just a dummy response:
    return {
        "pizza": {
            "type": "Space Pizza",
            "size": "Medium",
            "price": 13.99
        },
        "toppings": [
            {"name": "Alien Mushrooms", "price": 2.50},
            {"name": "Meteorite Crust", "price": 3.00}
        ],
        "drinks": [
            {"name": "Galactic Soda", "size": "Medium", "price": 4.00}
        ],
        "sides": [
            {"name": "Comet Fries", "size": "Large", "price": 5.00}
        ],
        "total_price": 28.49
    }

# Create the Panel layout for interaction
dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

# Display the dashboard for interaction
dashboard.servable()


Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Send Order!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?